In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import scale, StandardScaler
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("Hitters.csv")
df.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N


In [3]:
df.dropna(inplace=True)

df = pd.get_dummies(df, columns=["League","Division","NewLeague"])

df.drop(["League_N", "Division_W", "NewLeague_N"], axis=1, inplace=True)

df.replace(to_replace=False, value=0, inplace=True)
df.replace(to_replace=True, value=1, inplace=True)

df.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,Salary,League_A,Division_E,NewLeague_A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,632,43,10,475.0,0,0,0
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,880,82,14,480.0,1,0,1
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,200,11,3,500.0,0,1,0
4,321,87,10,39,42,30,2,396,101,12,48,46,33,805,40,4,91.5,0,1,0
5,594,169,4,74,51,35,11,4408,1133,19,501,336,194,282,421,25,750.0,1,0,1


In [4]:
X = df.drop("Salary", axis=1)
y = df[["Salary"]]
X.shape, y.shape

((263, 19), (263, 1))

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=16)

In [6]:
# Yapay sinir ağları homojen verilerde daha iyi performans gösterdiği için 
# eğitim ve test verilerini dönüştürüyoruz

In [7]:
scaler = StandardScaler()

In [8]:
X_train_scaled = scaler.fit_transform(X_train)

In [9]:
X_test_scaled = scaler.fit_transform(X_test)

In [10]:
mlp_model = MLPRegressor(verbose=True).fit(X_train_scaled, y_train)

Iteration 1, loss = 238423.41098235
Iteration 2, loss = 238382.26749132
Iteration 3, loss = 238341.18539716
Iteration 4, loss = 238300.22883344
Iteration 5, loss = 238259.34929241
Iteration 6, loss = 238218.63127376
Iteration 7, loss = 238178.12114373
Iteration 8, loss = 238137.66483397
Iteration 9, loss = 238097.23696911
Iteration 10, loss = 238056.76711983
Iteration 11, loss = 238016.26565418
Iteration 12, loss = 237975.62619320
Iteration 13, loss = 237934.81457977
Iteration 14, loss = 237893.95201678
Iteration 15, loss = 237853.00015298
Iteration 16, loss = 237811.85596139
Iteration 17, loss = 237770.59415773
Iteration 18, loss = 237729.25015839
Iteration 19, loss = 237687.86430355
Iteration 20, loss = 237646.37373421
Iteration 21, loss = 237604.73581392
Iteration 22, loss = 237562.95193137
Iteration 23, loss = 237520.97613149
Iteration 24, loss = 237478.74567107
Iteration 25, loss = 237436.18031414
Iteration 26, loss = 237393.33739311
Iteration 27, loss = 237350.17731158
Iteration 

In [11]:
mlp_model.get_params()

{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (100,),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 200,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': None,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': True,
 'warm_start': False}

In [42]:
y_pred = mlp_model.predict(X_test_scaled)

In [43]:
# rmse
np.sqrt(mean_squared_error(y_test, y_pred))

690.9618620729432

In [45]:
r2_score(y_test, y_pred)

-1.1802521593904012

In [14]:
# Model Tuning

In [35]:
mlp_params = {
    "alpha": np.logspace(-3, 0, 4),
    "hidden_layer_sizes": [(20,5), (20,10), (20,10,10)],
    "max_iter": np.arange(2000, 10000, 2500)
}

In [36]:
mlp_cv = GridSearchCV(MLPRegressor(), mlp_params, cv=5, n_jobs=-1, verbose=2).fit(X_train_scaled, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


In [37]:
mlp_cv.best_params_

{'alpha': 1.0, 'hidden_layer_sizes': (20, 10), 'max_iter': 2000}

In [52]:
# Final Model

In [38]:
mlp_tuned = MLPRegressor(alpha=1, hidden_layer_sizes=(20,10), max_iter=2000 ,verbose=True).fit(X_train_scaled, y_train)
#alpha: L2 normu, hiden_layer_sizes: katman ve nöron sayıları, max_iter: epoch(döngü) sayısı 

Iteration 1, loss = 237771.10240573
Iteration 2, loss = 237753.58827337
Iteration 3, loss = 237735.99087439
Iteration 4, loss = 237718.26147947
Iteration 5, loss = 237700.42917847
Iteration 6, loss = 237682.60361485
Iteration 7, loss = 237664.86819267
Iteration 8, loss = 237647.12716110
Iteration 9, loss = 237629.31226116
Iteration 10, loss = 237611.44693592
Iteration 11, loss = 237593.48020819
Iteration 12, loss = 237575.47424128
Iteration 13, loss = 237557.39649788
Iteration 14, loss = 237539.23404191
Iteration 15, loss = 237520.93159927
Iteration 16, loss = 237502.46094718
Iteration 17, loss = 237483.82019010
Iteration 18, loss = 237464.98280479
Iteration 19, loss = 237446.05366802
Iteration 20, loss = 237427.01896556
Iteration 21, loss = 237407.80915519
Iteration 22, loss = 237388.42820698
Iteration 23, loss = 237368.82300990
Iteration 24, loss = 237349.02655709
Iteration 25, loss = 237329.10564651
Iteration 26, loss = 237309.01525465
Iteration 27, loss = 237288.62242230
Iteration 

In [47]:
y_pred = mlp_tuned.predict(X_test_scaled)

In [48]:
np.sqrt(mean_squared_error(y_test, y_pred))

235.1382346958679

In [50]:
r2_score(y_test, y_pred)

0.7475094791831776